In [1]:
import simulate
import baseline
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#import pypots
from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import SAITS
from pypots.utils.metrics import cal_mae, cal_rmse, cal_mre

c:\Users\RUSHI\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# UCI HAR with baseline methods

In [2]:
train_directory = "D:/GitHub/ece209as_project/data/UCI_HAR_Dataset/train/Inertial_Signals/"

train_files = ["body_acc_x_train.txt", "body_acc_y_train.txt", "body_acc_z_train.txt", 
        "body_gyro_x_train.txt", "body_gyro_y_train.txt", "body_gyro_z_train.txt", 
        "body_acc_x_train.txt", "body_acc_y_train.txt", "body_acc_z_train.txt"]

train_df = []
train_y = pd.read_csv("D:/GitHub/ece209as_project/data/UCI_HAR_Dataset/train/y_train.txt", header=None)

test_directory = "D:/GitHub/ece209as_project/data/UCI_HAR_Dataset/test/Inertial_Signals/"

test_files = ["body_acc_x_test.txt", "body_acc_y_test.txt", "body_acc_z_test.txt", 
        "body_gyro_x_test.txt", "body_gyro_y_test.txt", "body_gyro_z_test.txt", 
        "body_acc_x_test.txt", "body_acc_y_test.txt", "body_acc_z_test.txt"]

test_df = []
test_y = pd.read_csv("D:/GitHub/ece209as_project/data/UCI_HAR_Dataset/test/y_test.txt", header=None)


for train_file in train_files:
    df = pd.read_csv(train_directory+train_file, delim_whitespace=True, header=None)
    train_df.append(np.array(df))

for test_file in test_files:
    df = pd.read_csv(test_directory+test_file, delim_whitespace=True, header=None)
    test_df.append(np.array(df))

train_X = np.array(train_df).reshape([len(train_y), 128, 9])
test_X = np.array(test_df).reshape([len(test_y), 128, 9])

In [3]:
# MCAR
dict = simulate.simulate_nan(train_X, 0.1, "MCAR")
X_intact = dict['X_init']
X_missing = dict['X_incomp']
X_mask = dict['mask']

In [4]:
# Zero-filling imputation

X_zero_imputed = baseline.zero_filling_imputation(X_missing)

result = []
result.append([cal_mae(X_intact, X_zero_imputed), cal_rmse(X_intact, X_zero_imputed), cal_mre(X_intact, X_zero_imputed)])

In [5]:
# Mean imputation

X_mean_imputed = baseline.mean_imputation(X_missing)
result.append([cal_mae(X_intact, X_mean_imputed), cal_rmse(X_intact, X_mean_imputed), cal_mre(X_intact, X_mean_imputed)])

In [6]:
# Median imputation

X_median_imputed = baseline.median_imputation(X_missing)
result.append([cal_mae(X_intact, X_median_imputed), cal_rmse(X_intact, X_median_imputed), cal_mre(X_intact, X_median_imputed)])

In [7]:
result = pd.DataFrame(result, columns = ["mae", "rmse", "mre"], index=["zero-filling imputation", "mean imputation", "median imputation"])

In [8]:
result

,mae,rmse,mre
zero-filling imputation,0.011605,0.075121,1.316129e-08
mean imputation,0.011699,0.074824,1.308314e-08
median imputation,0.011484,0.075620,1.277239e-08
